### Imports

In [11]:
import pandas as pd
from utils.vectorization import text_vectorization, pad_tokenizer, get_embedding_matrix
from utils.downloader import get_glove_vectors
from utils.parser import generate_input_text, generate_output_text
from utils.ploting import plot_embeddings
from utils.training import test_step, train_step
from models.embeddings_model import EmbeddingsModel
from models.database import Database
from models.job_query import JOBQuery
from models.decoder import Decoder
from models.encoder import Encoder
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from gensim.models import Word2Vec, KeyedVectors
from sklearn.manifold import TSNE
import random
import sys
import numpy as np
import string
import logging
import random
import matplotlib.pyplot as plt
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, GRU, Bidirectional
from keras.layers.embeddings import Embedding
# from keras.losses import SparseCategoricalCrossentropy
# from keras.metrics import SparseCategoricalAccuracy, SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import os
import time
import pickle
import json

### Generate training-test data

In [ ]:
dataset = "data/queries"
prefix="training"

In [4]:
cwd = os.getcwd()
files = os.listdir(os.path.join(cwd, *dataset.split("/")))

db = Database(collect_db_info=True)
column_array_index = []
for table, columns in db.tables_attributes.items():
    for column in columns:
        column_array_index.append(table + "_" + column)

# initialize all variables
raw_input_texts = []
raw_output_texts = []
input_texts = []
target_texts = []

with open("sql.log", "w", encoding='utf-8') as logf:
    for file in files:
        with open(dataset + "/" + file, "r") as f:
            queries = f.read().strip()
            for query in queries.split(";"):
                if len(query) == 0:
                    continue

                try:
                    query = query.replace('\n', '').strip()
                    raw_input_texts.append(query)
                    job_query = JOBQuery(query)
                    rows = db.explain_query(query)
                    raw_output_texts.append(json.dumps(rows))

                    input_text = generate_input_text(job_query.predicates, job_query.rel_lookup)
                    input_texts.append(input_text)
                    # add '\t' at start and '\n' at end of text.
                    target_text = generate_output_text(rows, job_query.rel_lookup)[:-1]
                    target_texts.append(target_text)
                except Exception as e:
                    logf.write("Failed to execute query {0}: {1}\n".format(str(query), str(e)))
                    db.conn.close()
                    db.conn = db.connect()
                    if len(input_texts) != len(target_texts):
                        input_texts.pop()
                    if len(raw_input_texts) != len(raw_output_texts):
                        raw_input_texts.pop()
                finally:
                    pass


Failed parse where statement:  cn.country_code ='[us]'
  AND ct.kind IS NOT NULL
  AND (ct.kind ='production companies'
       OR ct.kind = 'distributors')
  AND it1.info ='budget'
  AND it2.info ='bottom 10 rank'
  AND t.production_year >2000
  AND (t.title LIKE 'Birdemic%'
       OR t.title LIKE '%Movie%')
  AND t.id = mi.movie_id
  AND t.id = mi_idx.movie_id
  AND mi.info_type_id = it1.id
  AND mi_idx.info_type_id = it2.id
  AND t.id = mc.movie_id
  AND ct.id = mc.company_type_id
  AND cn.id = mc.company_id
  AND mc.movie_id = mi.movie_id
  AND mc.movie_id = mi_idx.movie_id
  AND mi.movie_id = mi_idx.movie_id
Failed parse where statement:  ct.kind = 'production companies'
  AND it.info = 'top 250 rank'
  AND mc.note NOT LIKE '%(as Metro-Goldwyn-Mayer Pictures)%'
  AND (mc.note LIKE '%(co-production)%')
  AND t.production_year >2010
  AND ct.id = mc.company_type_id
  AND t.id = mc.movie_id
  AND t.id = mi_idx.movie_id
  AND mc.movie_id = mi_idx.movie_id
  AND it.id = mi_idx.info_type

Save inputs & outputs

In [15]:
input_df = pd.DataFrame(input_texts, columns=['input_queries'])
output_df = pd.DataFrame(target_texts, columns=['output_queries'])
input_df.to_csv(f"data/{prefix}/input_data.csv", encoding='utf-8', sep=',')
output_df.to_cfsv(f"data/{prefix}/output_data.csv", encoding='utf-8', sep=',')

raw_input_df = pd.DataFrame(raw_input_texts, columns=['input_queries'])
raw_output_df = pd.DataFrame(raw_output_texts, columns=['output_queries'])
raw_input_df.to_csv(f"data/{prefix}/raw_input_data.csv", encoding='utf-8', sep=';')
raw_output_df.to_csv(f"data/{prefix}/raw_output_data.csv", encoding='utf-8', sep=';')

Load inputs & outputs

In [ ]:
input_texts = pd.read_csv(f'data/{prefix}/input_data.csv')
print(f"input_texts shape: {input_texts.shape}")
output_texts = pd.read_csv(f'data/{prefix}/output_data.csv')
print(f"output_texts shape: {output_texts.shape}")
raw_input_texts = pd.read_csv(f'data/{prefix}/raw_input_data.csv', sep=';')
print(f"raw_input_texts shape: {raw_input_texts.shape}")
raw_output_texts = pd.read_csv(f'data/{prefix}/raw_output_data.csv', sep=';')
print(f"raw_output_texts shape: {raw_output_texts.shape}")

Vectorize inputs & outputs

In [7]:
input_vectorizer, input_corpus = text_vectorization(input_df, ['input_queries'], (1, 1))
output_vectorizer, output_corpus = text_vectorization(output_df, ['output_queries'], (1, 3))

print("number of encoder words : ", len(input_vectorizer.vocabulary_.keys()))
print("number of decoder words : ", len(output_vectorizer.vocabulary_.keys()))

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\user\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
c:\users\user\documents\unipi\ml-query-optimization\venv\lib\site-packages\sklearn\feature_extraction\text.py:516: UserWarning: The parameter 'token_pattern' will not b

In [8]:
raw_input_vectorizer, raw_input_corpus = text_vectorization(raw_input_df, ['input_queries'], (1, 1))
raw_output_vectorizer, raw_output_corpus = text_vectorization(raw_output_df, ['output_queries'], (1, 3))

print("number of raw encoder words : ", len(raw_input_vectorizer.vocabulary_.keys()))
print("number of raw decoder words : ", len(raw_output_vectorizer.vocabulary_.keys()))

number of encoder words :  30
number of decoder words :  464


### Train embedding models

In [ ]:
glove_vectors = get_glove_vectors()

In [9]:
input_encoder = EmbeddingsModel()
input_encoder.build(input_corpus, glove_vectors)
output_encoder = EmbeddingsModel()
output_encoder.build(output_corpus, glove_vectors)

In [ ]:
raw_input_encoder = EmbeddingsModel()
raw_input_encoder.build(raw_input_corpus, glove_vectors)
raw_output_encoder = EmbeddingsModel()
raw_output_encoder.build(raw_output_corpus, glove_vectors)

Save embedding models

In [ ]:
!mkdir -p data/embedding_models/training
!mkdir -p data/embedding_models/testing

In [ ]:
filename = f'data/embedding_models/{prefix}/input_encoder'
input_encoder.model.save(filename)
filename = f'data/embedding_models/{prefix}/output_encoder'
output_encoder.model.save(filename)

filename = f'data/embedding_models/{prefix}/raw_input_encoder'
raw_input_encoder.model.save(filename)
filename = f'data/embedding_models/{prefix}/raw_output_encoder'
raw_output_encoder.model.save(filename)

Plot embedding models

In [ ]:
plot_embeddings(input_encoder, 27, f"Input {prefix} Encoder Embeddings")
plot_embeddings(output_encoder, 38, f"Output {prefix} Encoder Embeddings")

Load embedding models

In [ ]:
input_encoder = EmbeddingsModel()
input_encoder.model = Word2Vec.load(f'data/embedding_models/{prefix}/input_encoder')
output_encoder = EmbeddingsModel()
output_encoder.model = Word2Vec.load(f'data/embedding_models/{prefix}/output_encoder')

In [ ]:
raw_input_encoder = EmbeddingsModel()
raw_input_encoder.model = Word2Vec.load(f'data/embedding_models/{prefix}/raw_input_encoder')
raw_output_encoder = EmbeddingsModel()
raw_output_encoder.model = Word2Vec.load(f'data/embedding_models/{prefix}/raw_output_encoder')

### Tokenize inputs & outputs

In [ ]:
x_w2v_weights = input_encoder.model.wv.vectors
x_vocab_size, x_embedding_size = x_w2v_weights.shape
print("Input {} vocabulary size: {} - Embedding Dim: {}".format(prefix, x_vocab_size, x_embedding_size))

y_w2v_weights = output_encoder.model.wv.vectors
y_vocab_size, y_embedding_size = y_w2v_weights.shape
print("Output {} vocabulary size: {} - Embedding Dim: {}".format(prefix, y_vocab_size, y_embedding_size))

In [ ]:
if prefix == 'training':
    X = input_texts['input_queries'].values
    y = output_texts['output_queries'].values

    # tokenize inputs - outputs
    x_max_length, x_vocab, X_pad = pad_tokenizer(X)
    y_max_length, y_vocab, y_pad = pad_tokenizer(y)

    X_train_pad, X_test_pad, y_train_pad, y_test_pad = train_test_split(X_pad, y_pad, test_size=0.2, random_state=42)

    raw_X = raw_input_texts['input_queries'].values
    raw_y = raw_output_texts['output_queries'].values

    # tokenize inputs - outputs
    raw_x_max_length, raw_x_vocab, raw_X_pad = pad_tokenizer(raw_X)
    raw_y_max_length, raw_y_vocab, raw_y_pad = pad_tokenizer(raw_y)

    raw_X_train_pad, raw_X_test_pad, raw_y_train_pad, raw_y_test_pad = train_test_split(raw_X_pad, raw_y_pad, test_size=0.2, random_state=42)

else:
    X = input_texts['input_queries'].values
    y = output_texts['output_queries'].values

    x_max_length, x_vocab, X_train_pad = pad_tokenizer(X)
    y_max_length, y_vocab, y_train_pad = pad_tokenizer(y)

    raw_X = raw_input_texts['input_queries'].values
    raw_y = raw_output_texts['output_queries'].values

    # tokenize inputs - outputs
    raw_x_max_length, raw_x_vocab, raw_X_train_pad = pad_tokenizer(raw_X)
    raw_y_max_length, raw_y_vocab, raw_y_train_pad = pad_tokenizer(raw_y)

### Build Model

Constants

In [ ]:
EMBEDDING_DIM = 300
SOS_token = 0
EOS_token = 1
BATCH_SIZE = 16
EPOCHS = 100
LOG_EVERY = 50
PATIENCE = 5
WAIT = 0
BEST = np.Inf
ENCODER_BEST_WEIGHTS = None
DECODER_BEST_WEIGHTS = None
BEST_EPOCH = 0
STOPPED_EPOCH = 0

Metrics functions

In [ ]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()
val_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy()

Build datasets

In [ ]:
input_tensor = tf.convert_to_tensor(X_train_pad)
output_tensor = tf.convert_to_tensor(y_train_pad)

buffer_size = len(input_tensor)
dataset = tf.data.Dataset.from_tensor_slices((input_tensor, output_tensor)).shuffle(buffer_size)
dataset = dataset.batch(BATCH_SIZE)
steps_per_epoch = len(input_tensor) // BATCH_SIZE

input_tensor_val = tf.convert_to_tensor(X_test_pad)
output_tensor_val = tf.convert_to_tensor(y_test_pad)

buffer_size_val = len(input_tensor_val)
dataset_val = tf.data.Dataset.from_tensor_slices((input_tensor_val, output_tensor_val)).shuffle(buffer_size_val)
dataset_val = dataset_val.batch(BATCH_SIZE)
steps_per_epoch_val = len(input_tensor_val) // BATCH_SIZE

Create checkpoint

In [ ]:
!mkdir -p data/weights

In [ ]:
encoder = Encoder(len(x_vocab) + 1, get_embedding_matrix(x_vocab, input_encoder.model.wv), x_max_length)
decoder = Decoder(len(y_vocab) + 1, get_embedding_matrix(y_vocab, output_encoder.model.wv), y_max_length, 'concat')

optimizer = Adam(0.005)

checkpoint_dir = 'data/weights/chkp'
checkpoint = tf.train.Checkpoint(optimizer=optimizer, encoder=encoder, decoder=decoder)

### Train Model

In [ ]:
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

for e in range(1, EPOCHS):
  start_time = time.time()
  total_loss = 0.0
  total_accuracy = 0.0
  enc_hidden = encoder.init_hidden(BATCH_SIZE)

  for idx, (input_tensor, target_tensor) in enumerate(dataset.take(steps_per_epoch)):
      # print("idx: {0}, input_tensor shape: {1}, target_tensor shape: {2}".format(idx, input_tensor.shape,
      #                                                                            output_tensor.shape))
      batch_loss, batch_accuracy = train_step(input_tensor, target_tensor, enc_hidden, encoder, decoder, optimizer)
      total_loss += batch_loss
      total_accuracy += batch_accuracy

      if idx % LOG_EVERY == 0:
          print("Epochs: {} batch {}/{} | batch_accuracy: {:.4f} | batch_loss: {:.4f}".format(e, idx, steps_per_epoch, batch_accuracy, batch_loss))

  train_acc = total_accuracy / steps_per_epoch
  train_accuracies.append(train_acc)
  train_loss = total_loss / steps_per_epoch
  train_losses.append(train_loss)
  checkpoint.save(file_prefix = checkpoint_dir)
  print("Epoch: {} | Training accuracy over epoch: {:.4f} | Training loss over epoch: {:.4f}".format(e, float(train_acc), float(train_loss)))

  total_loss = 0.0
  total_accuracy = 0.0
  for idx, (input_tensor_val, target_tensor_val) in enumerate(dataset_val.take(steps_per_epoch_val)):
    batch_loss, batch_accuracy = test_step(input_tensor_val, target_tensor_val, enc_hidden, encoder, decoder)
    total_loss += batch_loss
    total_accuracy += batch_accuracy

  val_acc = total_accuracy / steps_per_epoch_val
  val_accuracies.append(val_acc)
  val_loss = total_loss / steps_per_epoch_val
  val_losses.append(val_loss)
  print("Epoch: {} | Validation acc: {:.4f} | Validation loss: {:.4f}".format(e, float(val_acc), float(val_loss)))
  print("Time taken: %.2fs" % (time.time() - start_time))

  # if e % 2 == 0:
      # print("Epochs: {}/{} | total_loss: {:.4f} | total_accuracy: {:.4f}".format(
      #     e, EPOCHS, total_loss / steps_per_epoch, total_accuracy / steps_per_epoch))
  #    print("Epochs: {}/{} | total_loss: {:.4f}".format(e, EPOCHS, total_loss / steps_per_epoch))

  # The early stopping strategy: stop the training if `val_loss` does not
  # decrease over a certain number of epochs.

  if np.less(val_loss, BEST):
    BEST = val_loss
    WAIT = 0
    BEST_EPOCH = e
    # Record the best weights if current results is better (less).
    ENCODER_BEST_WEIGHTS = encoder.get_weights()
    DECODER_BEST_WEIGHTS = decoder.get_weights()
  else:
    WAIT += 1
    if WAIT >= PATIENCE:
      STOPPED_EPOCH = e
      encoder.stop_training = True
      decoder.stop_training = True
      print(f"Early stopping activated! Restoring model weights from the end of the (best) epoch: {BEST_EPOCH}.")
      encoder.set_weights(ENCODER_BEST_WEIGHTS)
      decoder.set_weights(DECODER_BEST_WEIGHTS)
      checkpoint.save(file_prefix = checkpoint_dir)
      break

Save results

In [ ]:
with open("data/weights/results.pkl", "wb") as f:
  results = {
    'train_losses': train_losses,
    'train_accuracies': train_accuracies,
    'val_losses': val_losses,
    'val_accuracies': val_accuracies
  }
  pickle.dump(results, f)

### Results

Load results

In [ ]:
with open("data/weights/results.pkl", "rb") as f:
  results = pickle.load(f)
results.keys()

In [ ]:
train_losses = [float(loss) for loss in results['train_losses']]
train_accuracies = [float(loss) for loss in results['train_accuracies']]
val_losses = [float(loss) for loss in results['val_losses']]
val_accuracies = [float(loss) for loss in results['val_accuracies']]

Plot results

In [ ]:
fig1, ax1 = plt.subplots()

ax1.plot(train_losses, color="C0", label='train')
ax1.plot(val_losses, color="C1", label='validation')

ax1.grid(which="major", axis="both")
ax1.set_ylabel("loss")
ax1.set_xlabel("epoch")
ax1.set_title('Loss')
ax1.legend()

fig1.show()

fig2, ax2 = plt.subplots()

ax2.plot(train_accuracies, color="C0", label='train')
ax2.plot(val_accuracies, color="C1", label='validation')

ax2.grid(which="major", axis="both")
ax2.set_ylabel("accuracy")
ax2.set_xlabel("epoch")
ax2.set_title('Accuracy')
ax2.legend()

fig2.show()